<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Regulatory compliance and customer complaint analytics powered by ClearScape Analytics In-db functions, BYO-LLM, and GPU compute
  <br>
       <img id="teradata-logo" src="images/TeradataLogo.png" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<hr>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Use case</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For financial institutions, failure to comply with regulations can have severe financial implications and reputation damage.   Monitoring employee-client communications helps to prevent irresponsible behavior. In addition, it is also required to manage customer complaints efficiently
In this demo you will see how use power of ClearScape Analytics In-db functions, BYO-LLM and GPU compute to analyze text related to irresponsible behavior and semantic search on customer complaints.</p>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Demonstrations Overview</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstrations illustrate the end-to-end process of how organizations can utilize VantageCloud Lake <b>GPU-enabled Analytic Cluster</b> architecture to run open-source large language models at massive parallelism and scale, and then leverage these next-generation capabilities in ad-hoc analytics, development, and operational processing.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This notebook contains several high-level demonstrations that can be run together or individually, and are designed to illustrate;</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Container Management</b>.  Administrators can create and manage <b>secure, custom</b> runtime containers that will host any number of models and model artifacts to unlock GPU-augmented analytics</li>
    <li><b>Data Prep with Vector Embeddings</b>. Developers will use the Hugging Face BAAI/bge-small-en-v1.5 model to generate vector embeddings for two data sets:<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li><b>Consumer Complaints</b>. A subset of the Consumer Financial Protection Board Complaints <a href = 'https://catalog.data.gov/dataset/consumer-complaint-database'>database</a> data has been loaded into the VantageCloud Lake database for this demonstration</li><li><b>Semantic Search Terms</b>.  A user-defined list of natural language topics to use to perform a Semantic Search</li></ul></li>
    <li><b>Operationalization with interactive search</b>. Combine the data prep and transformation steps with powerful native <b>ClearScape Analytics</b> functions against the data sets to derive insights at scale.  These similarity functions can rapidly identify the semantic relationship between user input and a large corpus of complaints data</li>
    </ol>

<b style = 'font-size:28px;font-family:Arial;color:#00233C'>End-to-End workflow</b>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The high level process is as follows:</p>

<table style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'>
    <tr><td style = 'vertical-align:top' width = '40%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>The Data Scientist conducts analytics activities using his or her own python tools and packages of choice, then connects to VantageCloud Lake through teradataml client library and teradatasql python driver.</li>
                <br>      
                <li>Teradataml provides APIs to create and manage custom runtime environments; including custom libraries, dependencies, model artifacts, and scoring scripts.  The user can leverage these APIs to create one or many custom, dedicated environments to host their code.</li>
                <br>
                <li>The Data Scientist will then execute their pipeline that will;
                    <ul style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'><li>Call ClearScape Analytics functions on Compute Clusters (data prep, transformation, etc.)</li>
                        <li>Prepared data is passed to the python container running in parallel on cluster nodes.</li>
                        <li>Results (inference/predictions) are returned as "virtual" dataframes; where the data resides <b>in Vantage</b></li>
                    </ul></li>
                <br>
                <li>Worfklow can be operationalized using SQL, and results can be returned to common BI tools, persisted as part of an ETL process, or embedded in application code</li>
            </ol>
        </td><td><img src = 'images/BYOLLM_Overview.png' width = '600'></td></tr>
</table>
<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Python Package Installation</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If necessary, install required client packages for the demonstrations.  User may need to restart the Jupyter kernel after installation.</p> 

In [ ]:
%pip install -r requirements.txt

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Python Package Imports</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Standard practice to import required packages and libraries; execute this cell to import packages for Teradata automation as well as machine learning, analytics, utility, and data management packages.</p> 

In [ ]:
from teradataml import *
from oaf_utils import *
from teradatasqlalchemy.types import *
from time import sleep
import pandas as pd
import csv, sys, os, warnings
from os.path import expanduser
from collections import OrderedDict
import ipywidgets as widgets
from wordcloud import WordCloud, STOPWORDS

from IPython.display import clear_output , display as ipydisplay
import matplotlib.pyplot as plt
from itables import init_notebook_mode
import itables.options as opt

# Set display options for dataframes, plots, and warnings

opt.style="table-layout:auto;width:auto;float:left"
opt.columnDefs = [{"className": "dt-left", "targets": "_all"}]
init_notebook_mode(all_interactive=True)
%matplotlib inline
warnings.filterwarnings('ignore')
display.suppress_vantage_runtime_warnings = True

# load vars json
with open('vars.json', 'r') as f:
    session_vars = json.load(f)

# Database login information
host = session_vars['environment']['host']
username = session_vars['hierarchy']['users']['business_users'][1]['username']
password = session_vars['hierarchy']['users']['business_users'][1]['password']

# UES Authentication information
ues_url = session_vars['environment']['UES_URI']
configure.ues_url = ues_url
pat_token = session_vars['hierarchy']['users']['business_users'][1]['pat_token']
pem_file = session_vars['hierarchy']['users']['business_users'][1]['key_file']


compute_group = session_vars['hierarchy']['users']['business_users'][1]['compute_group']


# get the current python version to match deploy a custom container
python_version = str(sys.version_info[0]) + '.' + str(sys.version_info[1])
print(f'Using Python version {python_version} for user environment')


# Hugging Face model for the demo
model_name = 'BAAI/bge-small-en-v1.5'

# a list of required packages to install in the custom OAF container
# modify this if using different models or design patterns
pkgs = ['transformers',
        'torch',
        'sentencepiece',
        'pandas',
        'numpy'
        'sentence-transformers',
        'dill']

# container name - set here for easier notebook navigation
### User will also be asked to change it ###
oaf_name = 'embeddings_env_demo'
###########################

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Connect to the database</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>After connecting, check cluster status. Start it if necessary - note the cluster only needs to be running to execute the APPLY sections of the demo.</p> 

In [ ]:
# check for existing connection
eng = check_and_connect(host=host, username=username, password=password, compute_group = compute_group)
print(eng)

In [ ]:
# check cluster status
res = check_cluster_start(compute_group = compute_group)

<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Container Management
  <br>
    </p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Teradata Vantage Python Client Library provides simple, powerful methods for the creation and maintenance of custom Python runtime environments <b>in the VantageCloud environment</b> .  This allows practitioners complete control over the behavior and quality of their model performance and analytic accuracy running on the Analytic Cluster.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Custom environments are persistent.</b> Users only need to create these once and then can be saved, updated, or modified only as needed.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Container Management Process</b></p>
<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '40%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Create a unique User Environment based on available base images</li>
                <br>
                <li>Install libraries</li>
                <br>
                <li>Install models and additional user artifacts</li>
            </ol>
        </td>
        <td><img src = 'images/OAF_Env.png' width = '600'></td>
    </tr>
</table>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Connect to the Environment Service</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>To better support integration with Cloud Services and commong automation tools; the <b > User Environment Service</b> is accessed via RESTful APIs.  These APIs can be called directly or in the examples shown below that leverage the Python Package for Teradata (teradataml) methods.</p> 

In [ ]:
# check to see if there is a valid UES auth
# if not, authenticate
try:
    demo_env = get_env(oaf_name)
    print('Existing valid UES token')

except Exception as e:
    if '''NoneType' object has no attribute 'value''' in str(e) or '''Failed to execute get_env''' in str(e):
        if set_auth_token(ues_url = ues_url, username = username, pat_token = pat_token, pem_file = pem_file):
            print('UES Authentication successful')
        else:
            print('UES Authentication failed, check URL and account info')
        pass
    else:
        raise
    

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.  Create a Custom Container in Vantage</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If desired, the user can create a <b>new</b> custom environment by starting with a "base" image and customizing it.  The steps are:</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>List the available "base" images the system supports</li>
    <li>List any existing "custom" environments the user has created</li>
    <li>If there are no custom environments, then create a new one from a base image</li>
    </ul>

In [ ]:
# Create a new environment, or connect to an existing one


try:
    ipydisplay(list_user_envs())
except Exception as e:
    
    if str(e).find('No user environments found') > 0:
        print('No user environments found')
        pass
    else:
        raise

print('Use an existing environment, or create a new one:')
print(f'OAF Environment is set to {oaf_name}.')
print('Enter to accept, or input a new value.')
print('If the environment is not in the list, a new one will be created')
i = input()
if len(i) != 0:
    oaf_name = i
    print(f'OAF Environment is now {oaf_name}')

try:
    demo_env = create_env(env_name = oaf_name,
                      base_env = f'python_{python_version}',
                      desc = 'OAF Demo env for LLM')
except Exception as e:
    if str(e).find('same name already exists') > 0:
        print('Environment already exists, obtaining a reference to it')
        demo_env = get_env(oaf_name)
        pass
    elif 'Invalid value for base environment name' in str(e):
        print('Unsupported base environment version, using defaults')
        demo_env = create_env(env_name = oaf_name,
                      desc = 'OAF Demo env for LLM')
    else:
        raise

# Note create_env seems to be asynchronous - sleep a bit for it to register
sleep(5)

try:
    ipydisplay(list_user_envs())
except Exception as e:
    if str(e).find('No user environments found') > 0:
        print('No user environments found')
        pass
    else:
        raise

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.  Install Dependencies</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The second step in the customization process is to install Python package dependencies. This demonstration uses the Hugging Face <a href = 'https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2'>all-MiniLM-L6-v2</a> Sentence Transformer.  Since VantageCloud Lake Analytic Clusters are secured by default against unauthorized access to the outside network, the user can load the required libraries and model using teradataml methods:
</p> 

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>List the currently installed models and python libraries</li>
    <li><b>If necessary</b>, install any required packages</li>
    <li><b>If necessary</b>, install the pre-trained model.  This process takes several steps;
        <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li>Import and download the model</li>
            <li>Create a zip archive of the model artifacts</li>
            <li>Call the install_model() method to load the model to the container</li>
        </ol></li>
    </ul>

In [ ]:
ipydisplay(demo_env.models)

# just showing a sample here - remove .head(5) to see them all
ipydisplay(demo_env.libs.head(5))

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>A note on package versions</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The next demonstration makes use of the DataFrame apply() method, which automatically passes the python code to the Analytic Cluster.  As such, one needs to ensue the python package versions match.  dill and pandas are required, as is any additional libraries for the use case.
</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note</b> while not required for many OAF use cases, for this demo the required packages for the model execution must be installed in the local environment first.</p>

In [ ]:
# Install any Python add-ons needed by the script in the user environment
# Using option asynchronous=True for an asychronous execution of the statement.
# Note: Avoid asynchronous installation when batch-executing all notebook statements,
#       as execution will continue even without installation being complete.
#
# Can install by passing a list of packages/versions
#   Or 
# install using a requirements.txt file.

# For this demo, 
# this code block will collect the current user's package versions
# for installation into the container
# when using dataframe.apply(), it pandas and dill are required
# to reduce issues, match the version between client and container

# import these functions inside of a function namespace
def get_versions(pkgs):
    local_v_pkgs = []
    for p in pkgs:

        #fix up any hyphened package names
        p_fixed = p.replace('-', '_')

        #import the packages and append the strings to the list
        exec(f'''import {p_fixed}; local_v_pkgs.append('{p}==' + str({p_fixed}.__version__))''')
    return local_v_pkgs

v_pkgs = get_versions(pkgs)



# check to see if these packages need to be installed
# by comparing the len of the intersection of the list of required packages with the installed ones
if not len(set([x.split('==')[0] for x in pkgs]).intersection(demo_env.libs['name'].to_list())) == len(pkgs):
    
    # pass the list of packages - split off any extra info from the version property e.g., plus sign
    claim_id = demo_env.install_lib([x.split('+')[0] for x in v_pkgs], asynchronous=True)
else:
    print(f'All required packages are installed in the {oaf_name} environment')

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Monitor library installation status</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optionally - users can monitor the library installation status using the cell below:
</p> 

In [ ]:
# Check the status of installation using status() API.
# Create a loop here for demo purposes
try: 
    claim_id
    ipydisplay(demo_env.status(claim_id))
    stage = demo_env.status(claim_id)['Stage'].iloc[-1]
    while stage == 'Started':
        stage = demo_env.status(claim_id)['Stage'].iloc[-1]
        clear_output()
        ipydisplay(demo_env.status(claim_id))
        sleep(5)
except NameError:
    print('No installations to monitor')

    
# Verify the Python libraries have been installed correctly.
ipydisplay(demo_env.libs)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.  Download and install pretrained model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Open Analytics Framework containers do not have open access to the external network, which contributes to a very secure runtime environment.  As such, users will load pre-trained models using the below APIs.  For illustration purposes, the following code will check to see if the model archive exists locally and if it doesn't, will import and download it by creating a model object.  The archive will then be created and installed into the remote environment.
</p> 

In [ ]:
# check to see if the model needs to be downloaded/archived

# construct the file name for the model:
model_fname = 'models--' + model_name.replace('/', '--')

if not os.path.isfile(f'{model_fname}.zip'):

    from sentence_transformers import SentenceTransformer
    import shutil
    print('Creating Model Archive...')

    model = SentenceTransformer(model_name)
    shutil.make_archive(model_fname, 
                        format='zip', 
                        root_dir=f'{expanduser("~")}/.cache/huggingface/hub/{model_fname}/')
else:
    print('Local model archive exists.')

# check to see if the model is already installed
try:
    if demo_env.models.empty: # no models installed at all
        print('Installing Model...')
        claim_id = demo_env.install_model(model_path = f'{model_fname}.zip', asynchronous = True)
    elif not any(model_fname in x for x in demo_env.models['Model']): #see if model is there
        print('Installing Model...')
        claim_id = demo_env.install_model(model_path = f'{model_fname}.zip', asynchronous = True)
    else:
        print('Model already installed')
except Exception as e:
    if '''NoneType' object has no attribute 'empty''' in str(e):
        print('Installing Model...')
        claim_id = demo_env.install_model(model_path = f'{model_fname}.zip', asynchronous = True)
        pass
    else:
        raise

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Monitor model installation status</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optionally - users can monitor the model installation status using the cell below:
</p> 

In [ ]:
# Check the status of installation using status() API.
# Create a loop here for demo purposes
try: 
    claim_id
    ipydisplay(demo_env.status(claim_id))
    stage = demo_env.status(claim_id)['Stage'].iloc[-1]
    while stage != 'File Installed':
        stage = demo_env.status(claim_id)['Stage'].iloc[-1]
        clear_output()
        ipydisplay(demo_env.status(claim_id))
        sleep(5)
except NameError:
    print('No installations to monitor')

    
# Verify the model has been installed correctly.
demo_env.refresh()
ipydisplay(demo_env.models)

<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Conclusion - Environment Management</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The preceding demo showed how users can perform a <b>one-time</b> configuration task to prepare a custom environment for analytic processing at scale.  Once this configuration is complete, these containers can be re-used in ad-hoc development tasks, or used for operationalizing analytics in production.</p>

<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Data Prep with Vector Embeddings
  <br>
    </p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstration will utilize the custom container environment built in the prior demonstration that uses the <a href = 'https://huggingface.co/BAAI/bge-small-en-v1.5'> Hugging Face bge-small-en-v1.5</a> pre-trained model to create vector embeddings for the two data sets:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Consumer Complaints</b></li>
    <li><b>Search Topics</b></li>
    </ol>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Once complete, these embeddings will be stored in the database, which will allow for broad analysis of this data at scale using native <b>ClearScape Analytics</b> functions, and integration with operational processing</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Compute Clusters are <b>elastic and ephemeral</b>; they offer automated scale up and down based on demand, and can be turned on or off on a schedule or programmatically.  This allows for greater cost and resource efficiency, where users can turn the GPU-enable cluster on for the duration of the transformation job, then shut it down when the processing is complete.</p>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Data Preparation Workflow</b></p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '30%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Write a <b>local</b> function that leverages a pre-trained Language Model to create vector embeddings</li>
                <br>
                <li>Push this processing to the <b>GPU Analytic Cluster</b> for processing at scale</li>
                <br>
                <li><b>Optimize</b> the embedding function to run at scale to maximize GPU throughput</li>
                <br>
                <li><b>Execute batch embedding</b> to store a vector representation of all consumer complaints</li>
            </ol>
        </td>
        <td style = 'vertical-align:top'>
            <img src = 'images/local_remote_functions.png'>
        <br>
        <img src = 'images/optimize.png'></td>
    </tr>
</table>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Check connection</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Check database and UES connection</p> 

In [ ]:
# check for existing connection
eng = check_and_connect(host=host, username=username, password=password, compute_group = compute_group)
print(eng)
    

# check to see if there is a valid UES auth
# if not, authenticate
try:
    demo_env = get_env(oaf_name)

except Exception as e:
    if '''NoneType' object has no attribute 'value''' in str(e) or '''Failed to execute get_env''' in str(e): #UES auth expired/required
        if set_auth_token(ues_url = ues_url, username = username, pat_token = pat_token, pem_file = pem_file):
            print('UES Authentication successful')
            try:
                demo_env = get_env(oaf_name)
                pass
            except Exception as l:
                if f'''User environment '{oaf_name}' not found''' in str(l):
                    print('User environment not found')
                    pass
                else:
                    raise
        else:
            print('UES Authentication failed, check URL and account info')
        pass
    elif f'''User environment '{oaf_name}' not found''' in str(e):
        print('User environment not found')
        pass
    else:
        raise


# check cluster status
check_cluster_start(compute_group = compute_group)

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.  Create a client-side embedding function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The goal of this exercise is to create a client-side function which can be "pushed" to the analytic cluster for processing at scale.  There are a few minor enhancements here to improve performance and usability in this remote environment:</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Imports</b>.  By default, the teradataml python client will package the code, objects, and dependencies and serialize it to the analytic cluster.  Users can make this more efficient by staging these objects before hand (done in the first demo of this notebook).  <b>Important</b> - place the larger libraries (sentence_transformers and torch) inside the function so they won't be registered as new dependencies that need to be installed.</li>
    <li><b>GPU Drivers</b>.  Since this function will run both on the client (CPU) and cluster (GPU), place a conditional to check</li>
    </ul>


In [ ]:
def create_embeddings(row):
    from sentence_transformers import SentenceTransformer
    import torch
    
    model = SentenceTransformer('BAAI/bge-small-en-v1.5') #'sentence-transformers/all-MiniLM-L6-v2'
    
    # check for NVIDIA drivers
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # create series from the embeddings list that is returned
    # m_out = model.encode(row['InputText'], normalize_embeddings=True, device = 'cuda')
    # print(m_out)
    embeddings = model.encode(row['InputText'], device = device, convert_to_tensor=True) #normalize_embeddings=True,
    # print(s)
    #concat them together
    row = pd.concat([row, pd.Series(embeddings.cpu().numpy())], axis = 0)
    
    # return row
    return row

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Test the function on local data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Python pandas provides an apply() method to execute the function across all rows in the DataFrame</p>

In [ ]:
df = pd.DataFrame({'id':[1,2], 'InputText':['hello world', 'hello back']})


df.apply(create_embeddings, axis = 1)



<hr>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Push this function to the analytic cluster</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Similar to pandas, the teradataml package provides an apply() method which is called in a similar manner, except it runs <b>in parallel on</b> the cluster, leveraging the GPU infrastructure and the MPP processing capabilities of Vantage.</p>

    

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Inspect the Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Simple DataFrame methods to show the data.  A <b>teradataml DataFrame</b> behaves like a normal pandas DataFrame, with one significant difference in that it is a reference to data on the analytic database.  This allows developers to perform familiar data mangement operations on extremely large data sets as if the data is local.</p>

In [ ]:
tdf_comments = DataFrame('"demo_ofs"."CFPB_Complaints"')

ipydisplay(tdf_comments.to_pandas(num_rows = 1))

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Prepare to execute the apply method</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>When using the teradataml apply method,there are some differences in how data is passed in and out of the multiple runtime containers on the distributed nodes.  This offers a great deal of processing power, but also requires some additional considerations when calling the method:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Optional Return Types</b>.  If the data being returned by the function has different columns/data types than the input DataFrame, the user passes these as dictionary value to the keyword argument</li>
    <li><b>Data Preparation</b>.  Of course, data preparation and cleansing can be performed inside the embeddings function using python. However, the unique architecture of the <b>Teradata Vantage</b> analytic database, is that users can execute powerful native data preparation functions that will operate at extreme scale and performance</li>
    </ol>

In [ ]:

types_dict = OrderedDict({})
types_dict['id'] = BIGINT()
types_dict['InputText'] = VARCHAR(1024)

for i in range(384):
    key = '"' + str(i) + '"'
    types_dict[key] = FLOAT()

tdf = DataFrame.from_query('''SELECT TOP 10 id,
    CASE 
        WHEN txt IS NULL THEN ' '
        ELSE regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(txt , X'0d' , ' ') , X'0a' , ' ') , X'09', ' '), ',', ' '), '"', ' ')
    END InputText
    FROM demo_ofs.CFPB_Complaints;''')

tdf.head(2)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Execute the function on the nodes</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Call the apply method on the teradataml DataFrame.  This will push the function to the analytic nodes for processing in parallel.  Data is returned as another teradataml DataFrame, which represents the entire result set of the operation.  For this example, additional method arguments include:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Function</b>.  The name of the function to pass - this is the original client-side function written above</li>
    <li><b>Environment</b>.  The custom runtime environment with the models and dependencies loaded</li>
    <li><b>Return types</b>. OrderedDict that represents the column names and data types</li>
    </ol>

In [ ]:
# create a new dataframe representing the embeddings
tdf_embedded = tdf.apply(lambda row: create_embeddings(row),
                    env_name = demo_env, 
                    returns = types_dict)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Check the data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now - users can create a table with these embeddings, use it in additional analytics, or operationalize this as part of a pipeline.</p>

In [ ]:
ipydisplay(tdf_embedded.sample(2))

<hr>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3.  Create a server-side embedding function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The goal of this exercise is to create a <b>server-side</b> function which can be staged on the analytic cluster.  This offers many improvements over the method used above;</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Performance</b>.  Staging the code and dependencies in the container environment reduces the amount of I/O, since the function doesn't need to get serialized to the cluster when called</li>
    <li><b>Operationalization</b>.  The execution pipeline can be encapsulated into a SQL statement, which allows for seamless use in ETL pipelines, dashboards, or applications that need access</li>
    <li><b>Flexibility</b>. Developers can express much greater flexibility in how the code works to optimize for performance, stability, data cleanliness or flow logic</li>
</ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>These benefits do come with some amount of additional work.  Developers need to account how data is passed in and out of the code runtime, and how to pass it back to the SQL engine to assemble and return the final resultset.  Code is executed when the user expresses an <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/SQL-Reference/SQL-Operators-and-User-Defined-Functions/Table-Operators/APPLY'>APPLY SQL function</a>;</p> 
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Input Query</b>.  The APPLY function takes a SQL query as input.  This query can be as complex as needed and include data preparation, cleansing, and/or any other set-based logic necessary to create the desired input data set.  This complexity can also be abstracted into a database view.  When using the teradata client connectors for Python or R, thise query is represented as a DataFrame or tibble.</li>
    <li><b>Pre-processing</b>.  Based on the query plan, data is retrieved from storage (cache, block storage, or object storage) and the input query is executed.</li>
    <li><b>Distribution</b>.  Input data can be partitioned and/or ordered to be processed on a specific container or collection of them.  For example, the user may want to process all data for a single post code in one partition, and run thousands of these in parallel.  Data can also be distributed evenly across all units of parallelism in the system</li>
    <li><b>Input</b>.  The data for each container is passed to the runtime using tandard input (stdin)</li>
    <li><b>Processing</b>.  The user's code executes, parsing stdin for the input data</li>
    <li><b>Output</b>.  Data is sent out of the code block using standard output (stdout)</li>
    <li><b>Resultset</b>.  Resultset is assembled by the analytic database, and the SQL query returns</li>
    </ol>


<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Example server-side code block</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This is the python script used in the demonstration.  It is saved to the filesystem as "embedding.py".  Note here there have been multiple optimizations to improve performance at scale; specifically using batching as compared to dataframe.apply() which is much slower.</p> 


```python

#!/usr/bin/env python3
import sys, csv
import warnings
import torch 
from sentence_transformers import SentenceTransformer
import pandas as pd

warnings.simplefilter('ignore')


# 1. use the csv reader to parse comma-separated input
# 2. construct the Dataframe from the resulting dictionary

colNames = ['id', 'text']
d = csv.DictReader(sys.stdin.readlines(), fieldnames = colNames)
df = pd.DataFrame(d, columns = colNames)

# Use try...except to produce an error if something goes wrong in the try block
try:
    # Exit gracefully if DataFrame is empty
    # It is possible some partitions won't get any data
    if df.empty:
        sys.exit()
        
    
    model = SentenceTransformer('BAAI/bge-small-en-v1.5')
    #################################
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device) #<-- let's load the model once in the gpu (when gpu available)
    # I fear that in the previous code, the model was loaded for each rows
    #################################



# Read data from stdin, and construct a Pandas DataFrame #
# Data can also be read/processed directly from stdin i
    def create_embeddings(texts):
        # Encode texts using the model in batches
        with torch.no_grad(): # just to be sure, I am not sure if gradient computations are turned off by SentenceTransformer
            embeddings = model.encode(texts, device=device, batch_size=2048, convert_to_tensor=True)
        return embeddings.cpu().numpy() # here we go back to cpu
    ####################################################


    ########################################################
    # There is no need to use a apply and do row by row
    texts = df['text'].tolist()
    embeddings = create_embeddings(texts)
    # Now the embeddings are calculated, we concatenate them in the dataframe:
    embedding_cols = [f'V{i}' for i in range(embeddings.shape[1])]
    embedding_df = pd.DataFrame(embeddings, columns=embedding_cols)
    df = pd.concat([df[['id']], embedding_df], axis=1)
    

    # Egress results to the Database through standard output.
    # iterrrows generates a Series, iterate through the series to construct
    # a comma-separated output string
    for index, value in df.iterrows():
        my_str = ''
        for val in value.index:
            my_str = my_str + str(value[val]) + ','
        print(my_str[:-1])
        
# raise any errors back to the SQL engine
except (SystemExit):
    # Skip exception if system exit requested in try block
    pass
except:    # Specify in standard error any other error encountered
    print("Script Failure :", sys.exc_info()[0], file=sys.stderr)
    raise
    sys.exit()
```

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Install the file and any additional artifacts</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the install_file() method to install this python file to the container.  As a reminder, this container is persistent, so these steps need only be done infrequently.</p> 

In [ ]:
demo_env.install_file('embedding_bge_small_en.py', replace = True)

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Call the APPLY function </b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function can be executed in two ways;</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b><a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Teradata-Package-for-Python-on-VantageCloud-Lake/Working-with-Open-Analytics/teradataml-Apply-Class-for-APPLY-Table-Operator'>Python</a></b> by calling the Apply() module function</li>
    <li><b><a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/SQL-Reference/SQL-Operators-and-User-Defined-Functions/Table-Operators/APPLY'>SQL</a></b> which allows for broad adoption across the enterprise</li>
    </ul>
    

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>APPLY using Python</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The process is as follows</p> 
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Construct a dictionary that will define the return columns and data types</li>
    <li>Construct a teradataml DataFrame representing the data to be processed - note this is a "virtual" object representing data and logic <b>in-database</b></li>
    <li>Execute the module function.  This constructs the function call in the database, but does not execute anything.  Note the Apply function takes several arguments - the input data, environment name, and the command to run</li>
    <li>In order to execute the function, an "execute_script()" method must be called.  This method returns the server-side DataFrame representing the complete operation.  This DataFrame can be used in further processing, stored as a table, etc.</li>
    </ol>
    

In [ ]:
types_dict = OrderedDict({})
types_dict['id'] = BIGINT()

for i in range(384):
    key = '"' + str(i) + '"'
    types_dict[key] = FLOAT()
    


view_query = '''REPLACE VIEW Cleaned_Text_V AS
                SELECT id,
                    CASE 
                        WHEN txt IS NULL THEN ' '
                        ELSE regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(txt , X'0d' , ' ') , X'0a' , ' ') , X'09', ' '), ',', ' '), '"', ' ')
                    END text
                FROM demo_ofs.CFPB_Complaints;'''
execute_sql(view_query)

tdf = DataFrame('Cleaned_Text_V')

In [ ]:
apply_obj = Apply(data = tdf.iloc[:1000],
                  apply_command = 'python embedding_bge_small_en.py',
                  returns = types_dict,
                  env_name = oaf_name,
                 )

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Execute the function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Call execute_script(), and return a single record to the client to check the data.  Note there is some amount of startup time, since the container must start up and receive data.  These use cases get much more effective as the volume of the data being processed increases</p> 

In [ ]:
%%time
ipydisplay(apply_obj.execute_script().sample(1))

<hr>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Persist the resultset</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Simple methods for saving the results to a table for reuse.  The copy_to_sql() function will persist the contents of the DataFrame to a table in the database.  The performance optimizations used in the cluster-side embedding function will process records rapidly at scale.</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>On a single-node GPU cluster, this will take approximately 2 minutes to process all records</b></p>

In [ ]:
%%time
copy_to_sql(Apply(data = tdf,
                  apply_command = 'python embedding_bge_small_en.py',
                  returns = types_dict,
                  env_name = oaf_name
                 ).execute_script(), table_name = 'CFPB_embeddings', schema_name = 'demo_ofs', if_exists = 'replace')
execute_sql('SELECT COUNT(1) FROM demo_ofs.CFPB_embeddings;').fetchall()

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Construct a "topics" table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a small table of search topics, write it to the database.  Next, create an embeddings table out of those topics.</p> 

In [ ]:
df = pd.DataFrame({'id': [1,2,3,4,5,6],
      'txt': ['Fradulent activity with Debit Cards at Wells Fargo',
              'Identity theft issues at Citibank',
              'Multiple account openings without authorization',
              'Irresponsible behavior by customer support',
              'App issues when transacting with bank',
              'Cant get money out of ATM',
              ]})
print('Writing topics table to the database')
copy_to_sql(df,table_name='topics_of_interest', schema_name = 'demo_ofs', 
            if_exists='replace', index=False, types = OrderedDict({'id':BIGINT()}))

print('Creating embeddings table')
apply_obj = Apply(data = DataFrame('"demo_ofs"."topics_of_interest"'),
                  apply_command = 'python embedding_bge_small_en.py',
                  returns = types_dict,
                  env_name = oaf_name
                 )

copy_to_sql(apply_obj.execute_script(), table_name = 'topics_embeddings', schema_name = 'demo_ofs', if_exists = 'replace')

print('Embedding complete')
print('Sample of the data:')
ipydisplay(execute_sql('SELECT TOP 1 id, "0", "1" FROM demo_ofs.topics_embeddings;').fetchall())

<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Conclusion - executing custom code against GPU-enabled analytic clusters</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The preceding demo showed how users can use simple, familiar patters to execute powerful AI models <b>at scale</b> for development and operational processing.  This processing can be used in batch data preparation tasks, or operationalized as the below example will show.</p>

<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Operationalizing interactive AI-powered analytics
  <br>
    </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstration will illustrate how developers can take the next step in the process to <b>operationalize</b> this processing, enabling the entire organization to leverage AI across the data lifecycle, including</p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '30%'>
           <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
               <li><b>Calculate Semantic Similarity</b>.  Execute native vector distance functions to check similarity between the search topics and complaints embeddings</li>
            <br>
            <br>
               <li><b>Examine Results</b>.  Generate a data table and word cloud to check for other contextual patterns</li>
            <br>
            <br>
               <li><b>Create an interactive exerience</b>.  Create an operational pipeline to take interactive semantic search terms</li>
        </ol>
        </td>
        <td width = '20%'></td>
        <td style = 'vertical-align:top'><img src = 'images/cos_similarity.jpeg' width = 350 ></td>
    </tr>
</table>


<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Check connection</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Reconnect to the database, UES, and start cluster if necessary<get_context()/p> 

In [ ]:
# check for existing connection
eng = check_and_connect(host=host, username=username, password=password, compute_group = compute_group)
print(eng)
    

# check to see if there is a valid UES auth
# if not, authenticate
try:
    demo_env = get_env(oaf_name)

except Exception as e:
    if '''NoneType' object has no attribute 'value''' in str(e) or '''Failed to execute get_env''' in str(e): #UES auth expired/required
        if set_auth_token(ues_url = ues_url, username = username, pat_token = pat_token, pem_file = pem_file):
            print('UES Authentication successful')
            try:
                demo_env = get_env(oaf_name)
                pass
            except Exception as l:
                if f'''User environment '{oaf_name}' not found''' in str(l):
                    print('User environment not found')
                    pass
                else:
                    raise
        else:
            print('UES Authentication failed, check URL and account info')
        pass
    elif f'''User environment '{oaf_name}' not found''' in str(e):
        print('User environment not found')
        pass
    else:
        raise


# check cluster status
check_cluster_start(compute_group = compute_group)

<hr>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1.  Calculate semantic similarity</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>First, create a reference to the prepared data sets.  Next, execute the native <b>ClearScape Analytics</b> VectorDistance function.</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Database-Analytic-Functions/Model-Training-Functions/TD_VectorDistance'>TD_VectorDistance</a> function accepts a table of target vectors and a table of reference vectors and returns a table that contains the distance between target-reference pairs. The function computes the distance between the target pair and the reference pair from the same table if you provide only one table as the input.  This function can be called using the terdataml python package, or native SQL.</p>

In [ ]:
print('Complaints embeddings:')
tdf_target = DataFrame('"demo_ofs"."CFPB_embeddings"')
ipydisplay(tdf_target.sample(1))
print('Topics embeddings:')
tdf_reference = DataFrame('"demo_ofs"."topics_embeddings"')
ipydisplay(tdf_reference.sample(1))

print('Calculating vector distance')
feature_list = ['"' + str(i) + '"' for i in range(384)]
res_tdf = VectorDistance(target_data = tdf_target, 
                   reference_data = tdf_reference, 
                   target_id_column = 'id',
                   target_feature_columns = feature_list,
                   ref_id_column = 'id',
                   ref_feature_columns = feature_list,
                   distance_measure = 'cosine',
                   topk = 1
                  ).result
print('Analysis complete, raw output: ')
res_tdf

<hr>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2.  Create user-friendly results</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>First, use various teradataml methods to join the original comments data to the distance calculations.</p> 

In [ ]:
# Get DataFrame references to the original data sets
tdf_topics = DataFrame('"demo_ofs"."topics_of_interest"')
tdf_full = DataFrame.from_query('''SELECT id,
    CASE 
        WHEN txt IS NULL THEN ' '
        ELSE regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(txt , X'0d' , ' ') , X'0a' , ' ') , X'09', ' '), ',', ' '), '"', ' ')
    END text
    FROM demo_ofs.CFPB_Complaints;''')

print('Joining topics text to distance calculations')
tdf_final = res_tdf.join(tdf_topics, on = 'reference_id = id')
print('Joining comments text to distance calculations')
tdf_final = tdf_final.join(tdf_full, on = 'target_id = id', lsuffix = 'l')
tdf_final = tdf_final.assign(similarity = 1 - tdf_final['distance'])
tdf_final = tdf_final.assign(topic = tdf_final['txt'])
tdf_final = tdf_final.assign(comment = tdf_final['text'])
tdf_final = tdf_final.assign(topic_id = tdf_final['reference_id'])
tdf_final = tdf_final.drop(['target_id', 'reference_id', 'id_l', 'txt', 'text'], axis = 1)

print('Writing final results to a table')
copy_to_sql(tdf_final, table_name = 'similarity_results', if_exists = 'replace')
ipydisplay(DataFrame('similarity_results').head(2).to_pandas())

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Visualize results</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use client-side widgets to provide an interactive experience.  Return ten rows of selected results <b>only</b>.  Then, generate a Word Cloud to represent frequent terms.</p> 

In [ ]:
tdf_results = DataFrame('similarity_results')
topics_list = DataFrame('"demo_ofs"."topics_of_interest"').to_pandas()['txt'].to_list()

d = widgets.Dropdown(options=topics_list)

output = widgets.Output()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output()
            local_df = tdf_results[tdf_results['topic'] == change['new']].set_index('distance').iloc[:10].to_pandas().drop('topic', axis = 1)
            ipydisplay(local_df)
            text = " ".join(comment for comment in local_df['comment'])
            stopwords = set(STOPWORDS)
            wordcloud = WordCloud(stopwords = stopwords, width = 800, height=400, 
                                  max_words=100, min_word_length=1, 
                                  collocations = True, background_color = 'white').generate(text)
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud, interpolation='gaussian')
            plt.axis('off')
            plt.title(f'Word Cloud for Search Topic')
            plt.show()
            
d.observe(on_change)

print('Select Topic to Search:')
ipydisplay(d, output)


<hr>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3.  Interactive Semantic Search</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstration will illustrate an example of how to <b>operationalize</b> the combined Hugging Face embeddings, and native analytics performed in the steps above.  In this section, the python methods are replaced with SQL, which allows for broad adoption and re-use by many tools and applications.  Steps are as follow:</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Construct the SQL pipeline</b>.  This query will combine all the analytic steps above into a single expression:<ol style = 'font-size:16px;font-family:Arial;color:#00233C'><li>Embedding incoming search term using the Hugging Face model</li>
        <li>Passing the vector values to the TD_VectorDistance function</li>
        <li>Joining the distance measurements to the original comments</li>
        <li>Returning search results back to the user</li></ol></li>
    <li><b>User</b> Submits search term to the query</li>
    <li><b>Responses</b> are sent back to the client</li>
    </ul>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Embed incoming data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Note that each of these examples below executes this APPLY function.  There is a certain amount of overhead in launching the container and performing the inference.  It may be recommended that users embed the incoming data before passing it to SQL, which may improve interactive performance.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>APPLY using SQL</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The process is much the same as above when python was used, except the benefit is that this SQL can be used by a wide range of tools, applications, and dashboards; as well as automated processes.  Construct the statement using the following values:</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>
        <b>ON</b> Clause<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li>Input table or query</li>
        <li>Return columns and data types</li></ul></li>
    <li>Any partition column(s)</li>
    <li>The shell command to run</li>
    <li>Additional functional arguments</li>
    </ul>

In [ ]:
search_topic = input('Please Enter Search Term: ')
apply_qry = f'''
SELECT * FROM Apply(
    ON (SELECT 1 as id, '{search_topic}' as txt)
    PARTITION BY ANY
    
    returns(id BIGINT, "0" FLOAT, "1" FLOAT, "2" FLOAT, "3" FLOAT, "4" FLOAT, "5" FLOAT, "6" FLOAT, "7" FLOAT, "8" FLOAT, "9" FLOAT, "10" FLOAT, "11" FLOAT, "12" FLOAT, "13" FLOAT, "14" FLOAT, "15" FLOAT, "16" FLOAT, "17" FLOAT, "18" FLOAT, "19" FLOAT, "20" FLOAT, "21" FLOAT, "22" FLOAT, "23" FLOAT, "24" FLOAT, "25" FLOAT, "26" FLOAT, "27" FLOAT, "28" FLOAT, "29" FLOAT, "30" FLOAT, "31" FLOAT, "32" FLOAT, "33" FLOAT, "34" FLOAT, "35" FLOAT, "36" FLOAT, "37" FLOAT, "38" FLOAT, "39" FLOAT, "40" FLOAT, "41" FLOAT, "42" FLOAT, "43" FLOAT, "44" FLOAT, "45" FLOAT, "46" FLOAT, "47" FLOAT, "48" FLOAT, "49" FLOAT, "50" FLOAT, "51" FLOAT, "52" FLOAT, "53" FLOAT, "54" FLOAT, "55" FLOAT, "56" FLOAT, "57" FLOAT, "58" FLOAT, "59" FLOAT, "60" FLOAT, "61" FLOAT, "62" FLOAT, "63" FLOAT, "64" FLOAT, "65" FLOAT, "66" FLOAT, "67" FLOAT, "68" FLOAT, "69" FLOAT, "70" FLOAT, "71" FLOAT, "72" FLOAT, "73" FLOAT, "74" FLOAT, "75" FLOAT, "76" FLOAT, "77" FLOAT, "78" FLOAT, "79" FLOAT, "80" FLOAT, "81" FLOAT, "82" FLOAT, "83" FLOAT, "84" FLOAT, "85" FLOAT, "86" FLOAT, "87" FLOAT, "88" FLOAT, "89" FLOAT, "90" FLOAT, "91" FLOAT, "92" FLOAT, "93" FLOAT, "94" FLOAT, "95" FLOAT, "96" FLOAT, "97" FLOAT, "98" FLOAT, "99" FLOAT, "100" FLOAT, "101" FLOAT, "102" FLOAT, "103" FLOAT, "104" FLOAT, "105" FLOAT, "106" FLOAT, "107" FLOAT, "108" FLOAT, "109" FLOAT, "110" FLOAT, "111" FLOAT, "112" FLOAT, "113" FLOAT, "114" FLOAT, "115" FLOAT, "116" FLOAT, "117" FLOAT, "118" FLOAT, "119" FLOAT, "120" FLOAT, "121" FLOAT, "122" FLOAT, "123" FLOAT, "124" FLOAT, "125" FLOAT, "126" FLOAT, "127" FLOAT, "128" FLOAT, "129" FLOAT, "130" FLOAT, "131" FLOAT, "132" FLOAT, "133" FLOAT, "134" FLOAT, "135" FLOAT, "136" FLOAT, "137" FLOAT, "138" FLOAT, "139" FLOAT, "140" FLOAT, "141" FLOAT, "142" FLOAT, "143" FLOAT, "144" FLOAT, "145" FLOAT, "146" FLOAT, "147" FLOAT, "148" FLOAT, "149" FLOAT, "150" FLOAT, "151" FLOAT, "152" FLOAT, "153" FLOAT, "154" FLOAT, "155" FLOAT, "156" FLOAT, "157" FLOAT, "158" FLOAT, "159" FLOAT, "160" FLOAT, "161" FLOAT, "162" FLOAT, "163" FLOAT, "164" FLOAT, "165" FLOAT, "166" FLOAT, "167" FLOAT, "168" FLOAT, "169" FLOAT, "170" FLOAT, "171" FLOAT, "172" FLOAT, "173" FLOAT, "174" FLOAT, "175" FLOAT, "176" FLOAT, "177" FLOAT, "178" FLOAT, "179" FLOAT, "180" FLOAT, "181" FLOAT, "182" FLOAT, "183" FLOAT, "184" FLOAT, "185" FLOAT, "186" FLOAT, "187" FLOAT, "188" FLOAT, "189" FLOAT, "190" FLOAT, "191" FLOAT, "192" FLOAT, "193" FLOAT, "194" FLOAT, "195" FLOAT, "196" FLOAT, "197" FLOAT, "198" FLOAT, "199" FLOAT, "200" FLOAT, "201" FLOAT, "202" FLOAT, "203" FLOAT, "204" FLOAT, "205" FLOAT, "206" FLOAT, "207" FLOAT, "208" FLOAT, "209" FLOAT, "210" FLOAT, "211" FLOAT, "212" FLOAT, "213" FLOAT, "214" FLOAT, "215" FLOAT, "216" FLOAT, "217" FLOAT, "218" FLOAT, "219" FLOAT, "220" FLOAT, "221" FLOAT, "222" FLOAT, "223" FLOAT, "224" FLOAT, "225" FLOAT, "226" FLOAT, "227" FLOAT, "228" FLOAT, "229" FLOAT, "230" FLOAT, "231" FLOAT, "232" FLOAT, "233" FLOAT, "234" FLOAT, "235" FLOAT, "236" FLOAT, "237" FLOAT, "238" FLOAT, "239" FLOAT, "240" FLOAT, "241" FLOAT, "242" FLOAT, "243" FLOAT, "244" FLOAT, "245" FLOAT, "246" FLOAT, "247" FLOAT, "248" FLOAT, "249" FLOAT, "250" FLOAT, "251" FLOAT, "252" FLOAT, "253" FLOAT, "254" FLOAT, "255" FLOAT, "256" FLOAT, "257" FLOAT, "258" FLOAT, "259" FLOAT, "260" FLOAT, "261" FLOAT, "262" FLOAT, "263" FLOAT, "264" FLOAT, "265" FLOAT, "266" FLOAT, "267" FLOAT, "268" FLOAT, "269" FLOAT, "270" FLOAT, "271" FLOAT, "272" FLOAT, "273" FLOAT, "274" FLOAT, "275" FLOAT, "276" FLOAT, "277" FLOAT, "278" FLOAT, "279" FLOAT, "280" FLOAT, "281" FLOAT, "282" FLOAT, "283" FLOAT, "284" FLOAT, "285" FLOAT, "286" FLOAT, "287" FLOAT, "288" FLOAT, "289" FLOAT, "290" FLOAT, "291" FLOAT, "292" FLOAT, "293" FLOAT, "294" FLOAT, "295" FLOAT, "296" FLOAT, "297" FLOAT, "298" FLOAT, "299" FLOAT, "300" FLOAT, "301" FLOAT, "302" FLOAT, "303" FLOAT, "304" FLOAT, "305" FLOAT, "306" FLOAT, "307" FLOAT, "308" FLOAT, "309" FLOAT, "310" FLOAT, "311" FLOAT, "312" FLOAT, "313" FLOAT, "314" FLOAT, "315" FLOAT, "316" FLOAT, "317" FLOAT, "318" FLOAT, "319" FLOAT, "320" FLOAT, "321" FLOAT, "322" FLOAT, "323" FLOAT, "324" FLOAT, "325" FLOAT, "326" FLOAT, "327" FLOAT, "328" FLOAT, "329" FLOAT, "330" FLOAT, "331" FLOAT, "332" FLOAT, "333" FLOAT, "334" FLOAT, "335" FLOAT, "336" FLOAT, "337" FLOAT, "338" FLOAT, "339" FLOAT, "340" FLOAT, "341" FLOAT, "342" FLOAT, "343" FLOAT, "344" FLOAT, "345" FLOAT, "346" FLOAT, "347" FLOAT, "348" FLOAT, "349" FLOAT, "350" FLOAT, "351" FLOAT, "352" FLOAT, "353" FLOAT, "354" FLOAT, "355" FLOAT, "356" FLOAT, "357" FLOAT, "358" FLOAT, "359" FLOAT, "360" FLOAT, "361" FLOAT, "362" FLOAT, "363" FLOAT, "364" FLOAT, "365" FLOAT, "366" FLOAT, "367" FLOAT, "368" FLOAT, "369" FLOAT, "370" FLOAT, "371" FLOAT, "372" FLOAT, "373" FLOAT, "374" FLOAT, "375" FLOAT, "376" FLOAT, "377" FLOAT, "378" FLOAT, "379" FLOAT, "380" FLOAT, "381" FLOAT, "382" FLOAT, "383" FLOAT)
    USING
    
    APPLY_COMMAND('python embedding_bge_small_en.py')
    ENVIRONMENT('{oaf_name}')
    STYLE('csv')
    delimiter(',') 
) as d;'''

print(f'Vector Representation of the following statement:')
print('-------------------------')
print(search_topic)
print('-------------------------')
ipydisplay(execute_sql(apply_qry).fetchone()[0:4])
print('...')

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Calculate vector distance in-line</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Pass the results of the APPLY up to the TD_VectorDistance function.</p> 

In [ ]:
distance_qry = f'''
SELECT TOP 10 Target_ID, Distance, 1-Distance AS Similarity FROM TD_VectorDistance(
    ON demo_ofs.CFPB_embeddings AS TargetTable
    PARTITION BY ANY 
    ON ({apply_qry[:-1]}) AS ReferenceTable
    DIMENSION
    USING
    TargetIDColumn('id')
    TargetFeatureColumns('[1:384]')
    RefIDColumn('id')
    RefFeatureColumns('[1:384]')
    DistanceMeasure('cosine')
    TopK(1)
) as dst
ORDER BY Similarity DESC;'''

print(f'Top 10 Cosine Similarity Results for "{search_topic}":')
execute_sql(distance_qry).fetchall()

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Join original text data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Add projections and a JOIN clause to return readable results.</p> 

In [ ]:
join_qry = f'''

SELECT dt.Target_ID, cpl.txt, dt.Similarity
FROM demo_ofs.CFPB_Complaints cpl
 INNER JOIN ({distance_qry[:-1]}) dt
     ON dt.Target_ID = cpl.id;'''

print(f'Top semantic match for "{search_topic}": ')
execute_sql(join_qry).fetchone()

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Expand the SQL</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Show the full query.</p> 

In [ ]:
end_to_end = f'''
WITH aq AS (SELECT * FROM Apply(
    ON (SELECT 1 as id, '{search_topic}' as txt)
    PARTITION BY ANY
    
    returns(id BIGINT, "0" FLOAT, "1" FLOAT, "2" FLOAT, "3" FLOAT, "4" FLOAT, "5" FLOAT, "6" FLOAT, "7" FLOAT, "8" FLOAT, "9" FLOAT, "10" FLOAT, "11" FLOAT, "12" FLOAT, "13" FLOAT, "14" FLOAT, "15" FLOAT, "16" FLOAT, "17" FLOAT, "18" FLOAT, "19" FLOAT, "20" FLOAT, "21" FLOAT, "22" FLOAT, "23" FLOAT, "24" FLOAT, "25" FLOAT, "26" FLOAT, "27" FLOAT, "28" FLOAT, "29" FLOAT, "30" FLOAT, "31" FLOAT, "32" FLOAT, "33" FLOAT, "34" FLOAT, "35" FLOAT, "36" FLOAT, "37" FLOAT, "38" FLOAT, "39" FLOAT, "40" FLOAT, "41" FLOAT, "42" FLOAT, "43" FLOAT, "44" FLOAT, "45" FLOAT, "46" FLOAT, "47" FLOAT, "48" FLOAT, "49" FLOAT, "50" FLOAT, "51" FLOAT, "52" FLOAT, "53" FLOAT, "54" FLOAT, "55" FLOAT, "56" FLOAT, "57" FLOAT, "58" FLOAT, "59" FLOAT, "60" FLOAT, "61" FLOAT, "62" FLOAT, "63" FLOAT, "64" FLOAT, "65" FLOAT, "66" FLOAT, "67" FLOAT, "68" FLOAT, "69" FLOAT, "70" FLOAT, "71" FLOAT, "72" FLOAT, "73" FLOAT, "74" FLOAT, "75" FLOAT, "76" FLOAT, "77" FLOAT, "78" FLOAT, "79" FLOAT, "80" FLOAT, "81" FLOAT, "82" FLOAT, "83" FLOAT, "84" FLOAT, "85" FLOAT, "86" FLOAT, "87" FLOAT, "88" FLOAT, "89" FLOAT, "90" FLOAT, "91" FLOAT, "92" FLOAT, "93" FLOAT, "94" FLOAT, "95" FLOAT, "96" FLOAT, "97" FLOAT, "98" FLOAT, "99" FLOAT, "100" FLOAT, "101" FLOAT, "102" FLOAT, "103" FLOAT, "104" FLOAT, "105" FLOAT, "106" FLOAT, "107" FLOAT, "108" FLOAT, "109" FLOAT, "110" FLOAT, "111" FLOAT, "112" FLOAT, "113" FLOAT, "114" FLOAT, "115" FLOAT, "116" FLOAT, "117" FLOAT, "118" FLOAT, "119" FLOAT, "120" FLOAT, "121" FLOAT, "122" FLOAT, "123" FLOAT, "124" FLOAT, "125" FLOAT, "126" FLOAT, "127" FLOAT, "128" FLOAT, "129" FLOAT, "130" FLOAT, "131" FLOAT, "132" FLOAT, "133" FLOAT, "134" FLOAT, "135" FLOAT, "136" FLOAT, "137" FLOAT, "138" FLOAT, "139" FLOAT, "140" FLOAT, "141" FLOAT, "142" FLOAT, "143" FLOAT, "144" FLOAT, "145" FLOAT, "146" FLOAT, "147" FLOAT, "148" FLOAT, "149" FLOAT, "150" FLOAT, "151" FLOAT, "152" FLOAT, "153" FLOAT, "154" FLOAT, "155" FLOAT, "156" FLOAT, "157" FLOAT, "158" FLOAT, "159" FLOAT, "160" FLOAT, "161" FLOAT, "162" FLOAT, "163" FLOAT, "164" FLOAT, "165" FLOAT, "166" FLOAT, "167" FLOAT, "168" FLOAT, "169" FLOAT, "170" FLOAT, "171" FLOAT, "172" FLOAT, "173" FLOAT, "174" FLOAT, "175" FLOAT, "176" FLOAT, "177" FLOAT, "178" FLOAT, "179" FLOAT, "180" FLOAT, "181" FLOAT, "182" FLOAT, "183" FLOAT, "184" FLOAT, "185" FLOAT, "186" FLOAT, "187" FLOAT, "188" FLOAT, "189" FLOAT, "190" FLOAT, "191" FLOAT, "192" FLOAT, "193" FLOAT, "194" FLOAT, "195" FLOAT, "196" FLOAT, "197" FLOAT, "198" FLOAT, "199" FLOAT, "200" FLOAT, "201" FLOAT, "202" FLOAT, "203" FLOAT, "204" FLOAT, "205" FLOAT, "206" FLOAT, "207" FLOAT, "208" FLOAT, "209" FLOAT, "210" FLOAT, "211" FLOAT, "212" FLOAT, "213" FLOAT, "214" FLOAT, "215" FLOAT, "216" FLOAT, "217" FLOAT, "218" FLOAT, "219" FLOAT, "220" FLOAT, "221" FLOAT, "222" FLOAT, "223" FLOAT, "224" FLOAT, "225" FLOAT, "226" FLOAT, "227" FLOAT, "228" FLOAT, "229" FLOAT, "230" FLOAT, "231" FLOAT, "232" FLOAT, "233" FLOAT, "234" FLOAT, "235" FLOAT, "236" FLOAT, "237" FLOAT, "238" FLOAT, "239" FLOAT, "240" FLOAT, "241" FLOAT, "242" FLOAT, "243" FLOAT, "244" FLOAT, "245" FLOAT, "246" FLOAT, "247" FLOAT, "248" FLOAT, "249" FLOAT, "250" FLOAT, "251" FLOAT, "252" FLOAT, "253" FLOAT, "254" FLOAT, "255" FLOAT, "256" FLOAT, "257" FLOAT, "258" FLOAT, "259" FLOAT, "260" FLOAT, "261" FLOAT, "262" FLOAT, "263" FLOAT, "264" FLOAT, "265" FLOAT, "266" FLOAT, "267" FLOAT, "268" FLOAT, "269" FLOAT, "270" FLOAT, "271" FLOAT, "272" FLOAT, "273" FLOAT, "274" FLOAT, "275" FLOAT, "276" FLOAT, "277" FLOAT, "278" FLOAT, "279" FLOAT, "280" FLOAT, "281" FLOAT, "282" FLOAT, "283" FLOAT, "284" FLOAT, "285" FLOAT, "286" FLOAT, "287" FLOAT, "288" FLOAT, "289" FLOAT, "290" FLOAT, "291" FLOAT, "292" FLOAT, "293" FLOAT, "294" FLOAT, "295" FLOAT, "296" FLOAT, "297" FLOAT, "298" FLOAT, "299" FLOAT, "300" FLOAT, "301" FLOAT, "302" FLOAT, "303" FLOAT, "304" FLOAT, "305" FLOAT, "306" FLOAT, "307" FLOAT, "308" FLOAT, "309" FLOAT, "310" FLOAT, "311" FLOAT, "312" FLOAT, "313" FLOAT, "314" FLOAT, "315" FLOAT, "316" FLOAT, "317" FLOAT, "318" FLOAT, "319" FLOAT, "320" FLOAT, "321" FLOAT, "322" FLOAT, "323" FLOAT, "324" FLOAT, "325" FLOAT, "326" FLOAT, "327" FLOAT, "328" FLOAT, "329" FLOAT, "330" FLOAT, "331" FLOAT, "332" FLOAT, "333" FLOAT, "334" FLOAT, "335" FLOAT, "336" FLOAT, "337" FLOAT, "338" FLOAT, "339" FLOAT, "340" FLOAT, "341" FLOAT, "342" FLOAT, "343" FLOAT, "344" FLOAT, "345" FLOAT, "346" FLOAT, "347" FLOAT, "348" FLOAT, "349" FLOAT, "350" FLOAT, "351" FLOAT, "352" FLOAT, "353" FLOAT, "354" FLOAT, "355" FLOAT, "356" FLOAT, "357" FLOAT, "358" FLOAT, "359" FLOAT, "360" FLOAT, "361" FLOAT, "362" FLOAT, "363" FLOAT, "364" FLOAT, "365" FLOAT, "366" FLOAT, "367" FLOAT, "368" FLOAT, "369" FLOAT, "370" FLOAT, "371" FLOAT, "372" FLOAT, "373" FLOAT, "374" FLOAT, "375" FLOAT, "376" FLOAT, "377" FLOAT, "378" FLOAT, "379" FLOAT, "380" FLOAT, "381" FLOAT, "382" FLOAT, "383" FLOAT)
    USING
    
    APPLY_COMMAND('python embedding_bge_small_en.py')
    ENVIRONMENT('{oaf_name}')
    STYLE('csv')
    delimiter(',') 
) as d),

dt AS (SELECT TOP 10 Target_ID, Distance, 1-Distance AS Similarity FROM TD_VectorDistance(
    ON demo_ofs.CFPB_embeddings AS TargetTable
    PARTITION BY ANY 
    ON aq AS ReferenceTable
    DIMENSION
    USING
    TargetIDColumn('id')
    TargetFeatureColumns('[1:384]')
    RefIDColumn('id')
    RefFeatureColumns('[1:384]')
    DistanceMeasure('cosine')
    TopK(1)
) as dst
ORDER BY Similarity DESC)

SELECT dt.Target_ID, cpl.txt, dt.Similarity
FROM demo_ofs.CFPB_Complaints cpl
 INNER JOIN  dt
     ON dt.Target_ID = cpl.id;
'''
print(f'Top semantic match for "{search_topic}": ')
execute_sql(end_to_end).fetchone()

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Create an interactive search experience</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use this query in a dynamic UI - this could be used in a BI tool or other application.</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This first cell defines functions we will use with the Jupyter Widgets</p>

In [ ]:
def build_search(term, oaf_name):
    return f'''
WITH aq AS (SELECT * FROM Apply(
    ON (SELECT 1 as id, '{term}' as txt)
    PARTITION BY ANY
    
    returns(id BIGINT, "0" FLOAT, "1" FLOAT, "2" FLOAT, "3" FLOAT, "4" FLOAT, "5" FLOAT, "6" FLOAT, "7" FLOAT, "8" FLOAT, "9" FLOAT, "10" FLOAT, "11" FLOAT, "12" FLOAT, "13" FLOAT, "14" FLOAT, "15" FLOAT, "16" FLOAT, "17" FLOAT, "18" FLOAT, "19" FLOAT, "20" FLOAT, "21" FLOAT, "22" FLOAT, "23" FLOAT, "24" FLOAT, "25" FLOAT, "26" FLOAT, "27" FLOAT, "28" FLOAT, "29" FLOAT, "30" FLOAT, "31" FLOAT, "32" FLOAT, "33" FLOAT, "34" FLOAT, "35" FLOAT, "36" FLOAT, "37" FLOAT, "38" FLOAT, "39" FLOAT, "40" FLOAT, "41" FLOAT, "42" FLOAT, "43" FLOAT, "44" FLOAT, "45" FLOAT, "46" FLOAT, "47" FLOAT, "48" FLOAT, "49" FLOAT, "50" FLOAT, "51" FLOAT, "52" FLOAT, "53" FLOAT, "54" FLOAT, "55" FLOAT, "56" FLOAT, "57" FLOAT, "58" FLOAT, "59" FLOAT, "60" FLOAT, "61" FLOAT, "62" FLOAT, "63" FLOAT, "64" FLOAT, "65" FLOAT, "66" FLOAT, "67" FLOAT, "68" FLOAT, "69" FLOAT, "70" FLOAT, "71" FLOAT, "72" FLOAT, "73" FLOAT, "74" FLOAT, "75" FLOAT, "76" FLOAT, "77" FLOAT, "78" FLOAT, "79" FLOAT, "80" FLOAT, "81" FLOAT, "82" FLOAT, "83" FLOAT, "84" FLOAT, "85" FLOAT, "86" FLOAT, "87" FLOAT, "88" FLOAT, "89" FLOAT, "90" FLOAT, "91" FLOAT, "92" FLOAT, "93" FLOAT, "94" FLOAT, "95" FLOAT, "96" FLOAT, "97" FLOAT, "98" FLOAT, "99" FLOAT, "100" FLOAT, "101" FLOAT, "102" FLOAT, "103" FLOAT, "104" FLOAT, "105" FLOAT, "106" FLOAT, "107" FLOAT, "108" FLOAT, "109" FLOAT, "110" FLOAT, "111" FLOAT, "112" FLOAT, "113" FLOAT, "114" FLOAT, "115" FLOAT, "116" FLOAT, "117" FLOAT, "118" FLOAT, "119" FLOAT, "120" FLOAT, "121" FLOAT, "122" FLOAT, "123" FLOAT, "124" FLOAT, "125" FLOAT, "126" FLOAT, "127" FLOAT, "128" FLOAT, "129" FLOAT, "130" FLOAT, "131" FLOAT, "132" FLOAT, "133" FLOAT, "134" FLOAT, "135" FLOAT, "136" FLOAT, "137" FLOAT, "138" FLOAT, "139" FLOAT, "140" FLOAT, "141" FLOAT, "142" FLOAT, "143" FLOAT, "144" FLOAT, "145" FLOAT, "146" FLOAT, "147" FLOAT, "148" FLOAT, "149" FLOAT, "150" FLOAT, "151" FLOAT, "152" FLOAT, "153" FLOAT, "154" FLOAT, "155" FLOAT, "156" FLOAT, "157" FLOAT, "158" FLOAT, "159" FLOAT, "160" FLOAT, "161" FLOAT, "162" FLOAT, "163" FLOAT, "164" FLOAT, "165" FLOAT, "166" FLOAT, "167" FLOAT, "168" FLOAT, "169" FLOAT, "170" FLOAT, "171" FLOAT, "172" FLOAT, "173" FLOAT, "174" FLOAT, "175" FLOAT, "176" FLOAT, "177" FLOAT, "178" FLOAT, "179" FLOAT, "180" FLOAT, "181" FLOAT, "182" FLOAT, "183" FLOAT, "184" FLOAT, "185" FLOAT, "186" FLOAT, "187" FLOAT, "188" FLOAT, "189" FLOAT, "190" FLOAT, "191" FLOAT, "192" FLOAT, "193" FLOAT, "194" FLOAT, "195" FLOAT, "196" FLOAT, "197" FLOAT, "198" FLOAT, "199" FLOAT, "200" FLOAT, "201" FLOAT, "202" FLOAT, "203" FLOAT, "204" FLOAT, "205" FLOAT, "206" FLOAT, "207" FLOAT, "208" FLOAT, "209" FLOAT, "210" FLOAT, "211" FLOAT, "212" FLOAT, "213" FLOAT, "214" FLOAT, "215" FLOAT, "216" FLOAT, "217" FLOAT, "218" FLOAT, "219" FLOAT, "220" FLOAT, "221" FLOAT, "222" FLOAT, "223" FLOAT, "224" FLOAT, "225" FLOAT, "226" FLOAT, "227" FLOAT, "228" FLOAT, "229" FLOAT, "230" FLOAT, "231" FLOAT, "232" FLOAT, "233" FLOAT, "234" FLOAT, "235" FLOAT, "236" FLOAT, "237" FLOAT, "238" FLOAT, "239" FLOAT, "240" FLOAT, "241" FLOAT, "242" FLOAT, "243" FLOAT, "244" FLOAT, "245" FLOAT, "246" FLOAT, "247" FLOAT, "248" FLOAT, "249" FLOAT, "250" FLOAT, "251" FLOAT, "252" FLOAT, "253" FLOAT, "254" FLOAT, "255" FLOAT, "256" FLOAT, "257" FLOAT, "258" FLOAT, "259" FLOAT, "260" FLOAT, "261" FLOAT, "262" FLOAT, "263" FLOAT, "264" FLOAT, "265" FLOAT, "266" FLOAT, "267" FLOAT, "268" FLOAT, "269" FLOAT, "270" FLOAT, "271" FLOAT, "272" FLOAT, "273" FLOAT, "274" FLOAT, "275" FLOAT, "276" FLOAT, "277" FLOAT, "278" FLOAT, "279" FLOAT, "280" FLOAT, "281" FLOAT, "282" FLOAT, "283" FLOAT, "284" FLOAT, "285" FLOAT, "286" FLOAT, "287" FLOAT, "288" FLOAT, "289" FLOAT, "290" FLOAT, "291" FLOAT, "292" FLOAT, "293" FLOAT, "294" FLOAT, "295" FLOAT, "296" FLOAT, "297" FLOAT, "298" FLOAT, "299" FLOAT, "300" FLOAT, "301" FLOAT, "302" FLOAT, "303" FLOAT, "304" FLOAT, "305" FLOAT, "306" FLOAT, "307" FLOAT, "308" FLOAT, "309" FLOAT, "310" FLOAT, "311" FLOAT, "312" FLOAT, "313" FLOAT, "314" FLOAT, "315" FLOAT, "316" FLOAT, "317" FLOAT, "318" FLOAT, "319" FLOAT, "320" FLOAT, "321" FLOAT, "322" FLOAT, "323" FLOAT, "324" FLOAT, "325" FLOAT, "326" FLOAT, "327" FLOAT, "328" FLOAT, "329" FLOAT, "330" FLOAT, "331" FLOAT, "332" FLOAT, "333" FLOAT, "334" FLOAT, "335" FLOAT, "336" FLOAT, "337" FLOAT, "338" FLOAT, "339" FLOAT, "340" FLOAT, "341" FLOAT, "342" FLOAT, "343" FLOAT, "344" FLOAT, "345" FLOAT, "346" FLOAT, "347" FLOAT, "348" FLOAT, "349" FLOAT, "350" FLOAT, "351" FLOAT, "352" FLOAT, "353" FLOAT, "354" FLOAT, "355" FLOAT, "356" FLOAT, "357" FLOAT, "358" FLOAT, "359" FLOAT, "360" FLOAT, "361" FLOAT, "362" FLOAT, "363" FLOAT, "364" FLOAT, "365" FLOAT, "366" FLOAT, "367" FLOAT, "368" FLOAT, "369" FLOAT, "370" FLOAT, "371" FLOAT, "372" FLOAT, "373" FLOAT, "374" FLOAT, "375" FLOAT, "376" FLOAT, "377" FLOAT, "378" FLOAT, "379" FLOAT, "380" FLOAT, "381" FLOAT, "382" FLOAT, "383" FLOAT)
    USING
    
    APPLY_COMMAND('python embedding_bge_small_en.py')
    ENVIRONMENT('{oaf_name}')
    STYLE('csv')
    delimiter(',') 
) as d),

dt AS (SELECT TOP 10 Target_ID, Distance, 1-Distance AS Similarity FROM TD_VectorDistance(
    ON demo_ofs.CFPB_embeddings AS TargetTable
    PARTITION BY ANY 
    ON aq AS ReferenceTable
    DIMENSION
    USING
    TargetIDColumn('id')
    TargetFeatureColumns('[1:384]')
    RefIDColumn('id')
    RefFeatureColumns('[1:384]')
    DistanceMeasure('cosine')
    TopK(1)
) as dst
ORDER BY Similarity DESC)

SELECT dt.Target_ID, cpl.txt, dt.Similarity
FROM demo_ofs.CFPB_Complaints cpl
 INNER JOIN  dt
     ON dt.Target_ID = cpl.id
ORDER BY dt.Similarity DESC;
'''

def on_button_clicked(b):
    if len(t.value) > 0:
        with o:
            clear_output()
            print('Starting Search...')
            end_to_end = build_search(t.value, oaf_name)
            #print('Starting Search...')
            local_df = pd.read_sql(end_to_end, eng)[['target_id', 'Similarity', 'txt']]
            ipydisplay(local_df)
            text = " ".join(comment for comment in local_df['txt'])
            stopwords = set(STOPWORDS)
            wordcloud = WordCloud(stopwords = stopwords, width = 800, height=400, 
                                  max_words=100, min_word_length=1, 
                                  collocations = True, background_color = 'white').generate(text)
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud, interpolation='gaussian')
            plt.axis('off')
            plt.title(f'Word Cloud for Search Topic')
            plt.show()



<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Launch the UI</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Enter a term and click "Search".  Try misspellings to illustrate this is semantic vs. literal search.</p> 

In [ ]:
print('Enter Search Term: ')
t = widgets.Text(
    value='',
    placeholder='Type something')
b  = widgets.Button(
    description='Search',
    tooltip='Search')
o = widgets.Output()

ipydisplay(t, b, o)
b.on_click(on_button_clicked)

<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Conclusion - Operationalizing AI-powered analytics</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The preceding demo showed two methods for operationalizing the model execution; using python syntax, or embedding it as a simplified SQL view.  The former allows for developers and data scientists to easily embed this processing in their existing or new applications and workflows.  The latter allows for broad, democratized adoption across the data lifecycle and enterprise - enabling this analytic processing in ETL for data prep and transformation tasks, and in production to power dashboards and/or BI tools.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Cleanup</b></p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'><li>Stop the cluster</li>
    <li><b>Not required</b>, but if desired remove the environment and drop tables created in the demonstrations</li>
    <li>Disconnect from the database.</li>
    </ol>

In [ ]:
res = check_cluster_stop(compute_group)

In [ ]:
# uninstall the libraries from the environment first before removing it
# demo_env.uninstall_lib(libs = demo_env.libs['name'].to_list())
# remove_env(oaf_name)

In [ ]:
# Remove the tables created during this demo

# db_drop_table('similarity_results');
# db_drop_table('topics_of_interest', schema_name = 'demo_ofs');
# db_drop_table('topics_embeddings', schema_name = 'demo_ofs');
# db_drop_table('CFPB_embeddings', schema_name = 'demo_ofs')

In [ ]:
remove_context()